__Author:__ Bram Van de Sande

__Date:__ 8 FEB 2018

__Outline:__ This notebook clarifies the process by which the co-expression modules derived from GENIE3 can be refined into true regulomes (i.e. excluding indirect targets of transcription factors). Aka "RcisTarget".

In [1]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase, SQLiteRankingDatabase, MemoryDecorator
from pyscenic.genesig import GeneSignature, Regulome
from pyscenic.regulome import module2df_bincount_impl, derive_regulomes, module2df_numba_impl, module2regulome
from pyscenic.utils import load_motif_annotations

from dask import delayed
from dask.dot import dot_graph
from dask.multiprocessing import get
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from dask.diagnostics import ProgressBar
from distributed import LocalCluster, Client
from bokeh.io import output_notebook, push_notebook, show
output_notebook()
from dask.diagnostics import visualize

Loading BokehJS ...

In [2]:
%load_ext snakeviz
%load_ext line_profiler

In [3]:
DATA_FOLDER="/Users/bramvandesande/Projects/lcb/tmp"
RESOURCES_FOLDER="/Users/bramvandesande/Projects/lcb/resources"
DATABASE_FOLDER = "/Users/bramvandesande/Projects/lcb/databases/"

SQLITE_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.db")
FEATHER_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.feather")

MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")

NOMENCLATURE = "MGI"

Make databases in feather format are available.

In [4]:
if False:
    def derive_db_name(fname):
        return os.path.basename(fname).split(".")[0]

    from pyscenic.rnkdb import convert2feather
    
    for fname in glob.glob(SQLITE_GLOB):
        convert2feather(fname, DATABASE_FOLDER, derive_db_name(fname), NOMENCLATURE)

### Load resources

Co-expression modules were derived from GENIE3 output.

In [5]:
with open(os.path.join(DATA_FOLDER,'modules.pickle'), 'rb') as f:
    modules = pickle.load(f)

In [6]:
len(modules)

5106

### Load whole genome ranking databases

All implementations of the database are loaded for performance testing.

In [7]:
def name(fname):
    return os.path.basename(fname).split(".")[0]

In [8]:
db_fnames = glob.glob(FEATHER_GLOB)
dbs = [RankingDatabase(fname=fname, name=name(fname), nomenclature="MGI") for fname in db_fnames]

In [9]:
len(dbs)

6

In [10]:
sqldb_fnames = glob.glob(SQLITE_GLOB)
sqldbs = [SQLiteRankingDatabase(fname=fname, name=name(fname), nomenclature="MGI") for fname in sqldb_fnames]

In [11]:
len(sqldbs)

6

In [12]:
memdb = MemoryDecorator(dbs[0])

### Load motif annotations

In [13]:
motif_annotations = load_motif_annotations(MOTIF_ANNOTATIONS_FNAME)

In [14]:
motif_annotations.head()

,,motif_similarity_qvalue,orthologous_identity,description
gene_name,#motif_id,,,
Hoxa9,bergman__Abd-B,0.0006,1.000000,gene is annotated for similar motif cisbp__M10...
Zfp128,bergman__Aef1,0.0000,0.220264,motif is annotated for orthologous gene FBgn00...
Zfp853,bergman__Cf2,0.0000,0.166667,motif is annotated for orthologous gene FBgn00...
Nr1h2,bergman__EcR_usp,0.0000,0.378924,gene is orthologous to FBgn0000546 in D. melan...
Nr1h3,bergman__EcR_usp,0.0000,0.408989,gene is orthologous to FBgn0000546 in D. melan...


### Single-thread pipeline

Before scaling it via dask to work on the full combinatorial space of databases x modules.

In [15]:
%lprun -f module2regulome list((idx, module2regulome(dbs[0], module, motif_annotations, module2df_func=module2df_bincount_impl)) for idx, module in enumerate(modules[0:25]))

In [17]:
%%snakeviz
regulomes = list((idx, module2regulome(dbs[0], module, motif_annotations, module2df_func=module2df_bincount_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmpyziub1gk'. 


1. General performance is 78s for executing `module2regulome` 25 times.
1. 79% of time is spent at `recovery` and 18% at `db.load`.

#### SQLite-based storage + bincount implementation

In [18]:
%lprun -f module2regulome list((idx, module2regulome(sqldbs[0], module, motif_annotations, module2df_func=module2df_bincount_impl)) for idx, module in enumerate(modules[0:25]))

In [19]:
%%snakeviz
regulomes = list((idx, module2regulome(sqldbs[0], module, motif_annotations, module2df_func=module2df_bincount_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmpzsg2nvjf'. 


1. General performance is 83s for executing `module2regulome` 25 times.
1. 42% of time is spent at `recovery` and 56% at `db.load`.

#### In-memory database + bincount implementation

In [16]:
%lprun -f module2regulome list((idx, module2regulome(memdb, module, motif_annotations, module2df_func=module2df_bincount_impl)) for idx, module in enumerate(modules[0:25]))

In [23]:
%%snakeviz
regulomes = list((idx, module2regulome(memdb, module, motif_annotations, module2df_func=module2df_bincount_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmpfwp_6eat'. 


1. General performance is 78s for executing `module2regulome` 25 times.
1. 89% of time is spent at `recovery` and 8.5% at `db.load`.

#### In-memory database + bincount implementation: assess effect of reducing rank_threshold parameter

In [21]:
%lprun -f module2regulome list((idx, module2regulome(memdb, module, motif_annotations, auc_threshold=0.01, rank_threshold=750, module2df_func=module2df_bincount_impl)) for idx, module in enumerate(modules[0:25]))

1. General performance is 69s for executing `module2regulome` 25 times.
1. 93% of time is spent at `recovery` and 4.4% at `db.load`.

#### Feather-based storage + numba implementation

In [18]:
%%snakeviz
regulomes = list((idx, module2regulome(dbs[0], module, motif_annotations, module2df_func=module2df_numba_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmpslwt7n0d'. 


1. General performance is 49s for executing `module2regulome` 25 times.
1. 47% of time is spent at `recovery` and 24% at `db.load`.

#### Approach combining all potential improvements (in-memory database, auc-only calculation to assess enriched features and numba JIT implementation).

In [19]:
%%snakeviz
regulomes = list((idx, module2regulome(memdb, module, motif_annotations, module2df_func=module2df_numba_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmp2qfsqq87'. 


1. General performance is 39s for executing `module2regulome` 25 times.
1. 81% of time is spent at `recovery`

### Parallelized pipeline

#### Python multiprocessing implementation (db-dedicated workers using in memory copy + numba implementation of auc calculation).

Loading the database is also part of the overall timing. This will however dwarf when the number of modules increases.

In [10]:
%%timeit -n1 -r1 -o -q
regulomes = derive_regulomes(dbs[0:2], modules[0:50], MOTIF_ANNOTATIONS_FNAME, num_workers=4)
print(len(regulomes))

Using 4 workers.
Worker mm9-500bp-upstream-7species(2): database loaded in memory.
Worker mm9-500bp-upstream-7species(1): database loaded in memory.
Worker mm9-500bp-upstream-7species(2): motif annotations loaded in memory.
Worker mm9-500bp-upstream-7species(1): motif annotations loaded in memory.
Worker mm9-tss-centered-10kb-10species(2): database loaded in memory.
Worker mm9-tss-centered-10kb-10species(1): database loaded in memory.
Worker mm9-tss-centered-10kb-10species(2): motif annotations loaded in memory.
Worker mm9-tss-centered-10kb-10species(1): motif annotations loaded in memory.
Worker mm9-500bp-upstream-7species(1): Done.
Worker mm9-tss-centered-10kb-10species(1): Done.
Worker mm9-tss-centered-10kb-10species(2): Done.
Worker mm9-500bp-upstream-7species(2): Done.
16


<TimeitResult : 1min 18s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [10]:
regulomes = derive_regulomes(dbs[0:2], modules[0:50], MOTIF_ANNOTATIONS_FNAME, num_workers=4)

Using 4 workers.
Worker mm9-tss-centered-10kb-10species(1): database loaded in memory.
Worker mm9-tss-centered-10kb-10species(2): database loaded in memory.
Worker mm9-tss-centered-10kb-10species(1): motif annotations loaded in memory.
Worker mm9-tss-centered-10kb-10species(2): motif annotations loaded in memory.
Worker mm9-500bp-upstream-7species(1): database loaded in memory.
Worker mm9-500bp-upstream-7species(2): database loaded in memory.
Worker mm9-500bp-upstream-7species(1): motif annotations loaded in memory.
Worker mm9-500bp-upstream-7species(2): motif annotations loaded in memory.
Worker mm9-500bp-upstream-7species(1): Done.
Worker mm9-tss-centered-10kb-10species(1): Done.
Worker mm9-tss-centered-10kb-10species(2): Done.
Worker mm9-500bp-upstream-7species(2): Done.


In [11]:
regulomes

[Regulome(name='Regulome for Arntl', nomenclature='MGI', gene2weights=<frozendict {'Zfp668': 0, 'Bag6': 1, 'Rnf167': 18, 'Hmg20a': 14, 'Cog3': 22, 'Snrpb': 17, 'Pafah1b2': 25, 'Eif3i': 33, 'Vps52': 33, 'Trmt112': 29, 'Psma6': 1278, 'Dnaja2': 76, 'Cdk13': 59, 'Cpsf3': 43, 'Cstf2t': 83, 'Ncstn': 85, 'Fam188a': 48, 'Tmem161b': 49, 'Emg1': 51, 'Mtmr4': 63, 'Cap1': 160, 'Epn1': 141, 'Ttpal': 62, 'Nipbl': 86, 'Zfp410': 89, 'Krr1': 69, 'Rpl37': 87, 'Dalrd3': 71, 'Ebna1bp2': 914, 'Lrrc49': 173, 'Pick1': 381, 'Ngrn': 102, 'Uggt1': 86, 'Psmd13': 88, 'Zfp384': 447, 'Sptlc2': 167, 'Gatad1': 364, 'B3gat3': 101, 'Tmcc1': 780, 'Ppme1': 181, 'Rhot2': 167, 'Fbxw2': 190, 'Dctn5': 112, 'Ndufs3': 199, 'Pgap3': 119, 'Trim46': 434, 'Pfn1': 129, 'Lyrm1': 231, 'Ddost': 238, 'Ovca2': 400, 'Clasp1': 617, 'Vma21': 175, 'Zfp622': 405, 'Exoc1': 242, 'Tomm6': 173, 'Zfp3': 402, 'Erh': 204, 'Txnl4b': 530, 'Prpf19': 309, 'Ubl7': 558, '1110001J03Rik': 283, 'Lsm1': 214, 'Fmr1': 450, 'Rars': 466, 'Crk': 568, 'Arid4a': 49

In [ ]:
df = derive_regulomes(dbs[0:2], modules[0:50], MOTIF_ANNOTATIONS_FNAME, output="df", num_workers=4)

Using 4 workers.
Worker mm9-500bp-upstream-7species(1): database loaded in memory.
Worker mm9-500bp-upstream-7species(2): database loaded in memory.
Worker mm9-tss-centered-10kb-10species(2): database loaded in memory.
Worker mm9-tss-centered-10kb-10species(1): database loaded in memory.
Worker mm9-500bp-upstream-7species(1): motif annotations loaded in memory.
Worker mm9-500bp-upstream-7species(2): motif annotations loaded in memory.
Worker mm9-tss-centered-10kb-10species(2): motif annotations loaded in memory.
Worker mm9-tss-centered-10kb-10species(1): motif annotations loaded in memory.


Process mm9-tss-centered-10kb-10species(2):
Traceback (most recent call last):
  File "/Users/bramvandesande/miniconda3/envs/pyscenic_dev/lib/python3.6/site-packages/pandas/core/internals.py", line 4624, in create_block_manager_from_blocks
    mgr = BlockManager(blocks, axes)
  File "/Users/bramvandesande/miniconda3/envs/pyscenic_dev/lib/python3.6/site-packages/pandas/core/internals.py", line 3033, in __init__
    self._verify_integrity()
  File "/Users/bramvandesande/miniconda3/envs/pyscenic_dev/lib/python3.6/site-packages/pandas/core/internals.py", line 3244, in _verify_integrity
    construction_error(tot_items, block.shape[1:], self.axes)
  File "/Users/bramvandesande/miniconda3/envs/pyscenic_dev/lib/python3.6/site-packages/pandas/core/internals.py", line 4608, in construction_error
    passed, implied))
ValueError: Shape of passed values is (7150, 507), indices imply (7150, 15)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):

Worker mm9-500bp-upstream-7species(1): Done.


ValueError: Shape of passed values is (4671, 501), indices imply (4671, 4)


In [13]:
len(df)

306

In [14]:
df.head()

AUC       NES  \
gene_name #motif_id                                               
Arntl     dbcorrdb__ATF3__ENCSR000BJY_1__m1  0.042758  3.839269   
          dbcorrdb__ATF3__ENCSR000BKC_1__m1  0.040735  3.200513   
          dbcorrdb__ATF3__ENCSR000BKE_1__m1  0.041373  3.402032   
          dbcorrdb__ATF3__ENCSR000DOG_1__m1  0.045212  4.614271   
Atf2      cisbp__M0265                       0.039448  3.092005   

                                             motif_similarity_qvalue  \
gene_name #motif_id                                                    
Arntl     dbcorrdb__ATF3__ENCSR000BJY_1__m1                 0.000721   
          dbcorrdb__ATF3__ENCSR000BKC_1__m1                 0.000201   
          dbcorrdb__ATF3__ENCSR000BKE_1__m1                 0.000466   
          dbcorrdb__ATF3__ENCSR000DOG_1__m1                 0.000978   
Atf2      cisbp__M0265                                      0.000032   

                                             orthologous_identity  \
gene_name #motif_id                                                 
Arntl     dbcorrdb__ATF3__ENCSR000BJY_1__m1               1.00000   
          dbcorrdb__ATF3__ENCSR000BKC_1__m1               0.51816   
          dbcorrdb__ATF3__ENCSR000BKE_1__m1               0.51816   
          dbcorrdb__ATF3__ENCSR000DOG_1__m1               0.51816   
Atf2      cisbp__M0265                                    1.00000   

                                                                                   description  \
gene_name #motif_id                                                                              
Arntl     dbcorrdb__ATF3__ENCSR000BJY_1__m1  motif similar to homer__GNCACGTG_BMAL1 ('BMAL1...   
          dbcorrdb__ATF3__ENCSR000BKC_1__m1  motif similar to transfac_pro__M07637 ('I$CYC_...   
          dbcorrdb__ATF3__ENCSR000BKE_1__m1  motif similar to transfac_pro__M07637 ('I$CYC_...   
          dbcorrdb__ATF3__ENCSR000DOG_1__m1  motif similar to transfac_pro__M07637 ('I$CYC_...   
Atf2      cisbp__M0265                       gene is annotated for similar motif homer__NRR...   

                                                TF  \
gene_name #motif_id                                  
Arntl     dbcorrdb__ATF3__ENCSR000BJY_1__m1  Arntl   
          dbcorrdb__ATF3__ENCSR000BKC_1__m1  Arntl   
          dbcorrdb__ATF3__ENCSR000BKE_1__m1  Arntl   
          dbcorrdb__ATF3__ENCSR000DOG_1__m1  Arntl   
Atf2      cisbp__M0265                        Atf2   

                                                                                       context  
gene_name #motif_id                                                                             
Arntl     dbcorrdb__ATF3__ENCSR000BJY_1__m1  (target weight >= 0.001, mm9-tss-centered-10kb...  
          dbcorrdb__ATF3__ENCSR000BKC_1__m1  (target weight >= 0.001, mm9-tss-centered-10kb...  
          dbcorrdb__ATF3__ENCSR000BKE_1__m1  (target weight >= 0.001, mm9-tss-centered-10kb...  
          dbcorrdb__ATF3__ENCSR000DOG_1__m1  (target weight >= 0.001, mm9-tss-centered-10kb...  
Atf2      cisbp__M0265                       (target weight >= 0.001, mm9-tss-centered-10kb...

#### Dask framework using multiprocessing

In [18]:
with ProgressBar():
    with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
        regulomes = derive_regulomes(dbs[0:2], modules[0:50], MOTIF_ANNOTATIONS_FNAME,
                                     client_or_address="dask_multiprocessing")

[########################################] | 100% Completed |  2min  4.7s


In [19]:
len(regulomes)

16

In [20]:
visualize([prof, rprof, cprof])

Column(id='f87cf4af-d389-4ad3-9220-22a2c5c50d7a', ...)

#### Dask framework with custom client

In [15]:
local_cluster = LocalCluster(n_workers=6, 
                             threads_per_worker=1)

custom_client = Client(local_cluster)

In [16]:
custom_client

Client Scheduler: tcp://127.0.0.1:51083 Dashboard: http://127.0.0.1:8787,Cluster Workers: 6 Cores: 6 Memory: 12.88 GB


In [17]:
regulomes = derive_regulomes(dbs[0:2],
                             modules[0:50],
                             MOTIF_ANNOTATIONS_FNAME,
                             client_or_address=custom_client)

In [19]:
regulomes

<Future: status: finished, type: list, key: list_of_filter-420560af-d732-480f-b278-68c9b5468ba1>

In [21]:
regulomes.result()

[Regulome(name='Regulome for Arntl', nomenclature='MGI', gene2weights=<frozendict {'Zfp668': 1.0, 'Bag6': 1.0, 'Rnf167': 1.0, 'Hmg20a': 1.0, 'Cog3': 1.0, 'Snrpb': 1.0, 'Pafah1b2': 1.0, 'Eif3i': 1.0, 'Vps52': 1.0, 'Trmt112': 1.0, 'Psma6': 1.0, 'Dnaja2': 1.0, 'Cdk13': 1.0, 'Cpsf3': 1.0, 'Cstf2t': 1.0, 'Ncstn': 1.0, 'Fam188a': 1.0, 'Tmem161b': 1.0, 'Emg1': 1.0, 'Mtmr4': 1.0, 'Cap1': 1.0, 'Epn1': 1.0, 'Ttpal': 1.0, 'Nipbl': 1.0, 'Zfp410': 1.0, 'Krr1': 1.0, 'Rpl37': 1.0, 'Dalrd3': 1.0, 'Ebna1bp2': 1.0, 'Lrrc49': 1.0, 'Pick1': 1.0, 'Ngrn': 1.0, 'Uggt1': 1.0, 'Psmd13': 1.0, 'Zfp384': 1.0, 'Sptlc2': 1.0, 'Gatad1': 1.0, 'B3gat3': 1.0, 'Tmcc1': 1.0, 'Ppme1': 1.0, 'Rhot2': 1.0, 'Fbxw2': 1.0, 'Dctn5': 1.0, 'Ndufs3': 1.0, 'Pgap3': 1.0, 'Trim46': 1.0, 'Pfn1': 1.0, 'Lyrm1': 1.0, 'Ddost': 1.0, 'Ovca2': 1.0, 'Clasp1': 1.0, 'Vma21': 1.0, 'Zfp622': 1.0, 'Exoc1': 1.0, 'Tomm6': 1.0, 'Zfp3': 1.0, 'Erh': 1.0, 'Txnl4b': 1.0, 'Prpf19': 1.0, 'Ubl7': 1.0, '1110001J03Rik': 1.0, 'Lsm1': 1.0, 'Fmr1': 1.0, 'Rars': 1

In [22]:
custom_client.close()
local_cluster.close()